In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = 'https://www.numbeo.com/cost-of-living/'

# 1. Sahifani yuklab olamiz
headers = {'User-Agent': 'Mozilla/5.0'}  # Ba'zi saytlar headers talab qiladi
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# 2. Class nomini yozamiz
country_list_div = soup.find('div', class_='small_font links_for_countries')

# Tekshiruv
if country_list_div is None:
    print("Qaytadan urinib ko'ring")
else:
    # 3. Barcha <a> teglarini ajratib olamiz
    links = country_list_div.find_all('a')

    # 4. Natijani chiqaramiz
    for link in links:
        country_name = link.text.strip()
        country_url = 'https://www.numbeo.com' + link.get('href')



choosen_country = input("Davlat nomini kiriting: ").strip().lower()
for link in links:
    country_name = link.text.strip()
    if country_name.lower() == choosen_country:
        country_url = 'https://www.numbeo.com/cost-of-living/' + link.get('href')
        

# 5. Davlat sahifasini yuklab olish
response_child = requests.get(country_url, headers=headers)
soup_child = BeautifulSoup(response_child.text, 'html.parser')

# 6. Jadvalni topish (div emas, table!)
table = soup_child.find('table', class_='data_wide_table new_bar_table')

if table is None:
    print("❌ Jadval topilmadi — ehtimol sayt sahifasi himoyalangan yoki class nomi o‘zgargan.")
    print("DEBUG sahifa URL:", country_url)
    exit()


table_big = []
# 7. Jadval satrlarini ajratish
rows = table.find_all('tr')
for row in rows:
    cols = [col.text.strip() for col in row.find_all(['th', 'td'])]
    table_big.append(cols)


df = pd.DataFrame(table_big)
df.to_excel(f'{choosen_country}.xlsx', index=False)
